## Project LAB2 notebook

Importing the fasta sequences downloaded from PDB as a DataFrame

In [56]:
import pandas as pd
import random
import numpy as np
from Bio.PDB.PDBList import *
import os 
from Bio.PDB.DSSP import dssp_dict_from_pdb_file
from Bio.Blast.Applications import NcbipsiblastCommandline
import math
from sklearn import svm
import statistics
import joblib
from sklearn.metrics import confusion_matrix

## Parsing procedure

Once downloaded the fasta sequences in *csv* format from PDB I can conver and filter the sequences using the dataframe data structures

In [6]:
fasta_df=pd.read_csv("fasta_files.csv")
fasta_df

,Entry ID,Entity ID,Chain ID,Database Name,Accession Code(s),Sequence,Chain Length,Entity Macromolecule Type,Molecular Weight (Entity),Unnamed: 9
0,5D13,1,"A, B, C, D",UniProt,P31016,GSPEFLGEEDIPREPRRIVIHRGSTGLGFNIVGGEDGEGIFISFIL...,119,polypeptide(L),12.738,NaN
1,NaN,2,"E, F, G, H",NaN,NaN,XKKETEV,7,polypeptide(L),0.939,NaN
2,5COQ,1,"A, B, C, D",UniProt,P9WGR1,MGSSHHHHHHSSGLVPRGSHMTGLLDGKRILVSGIITDSSIAFHIA...,289,polypeptide(L),30.698,NaN
3,5CP5,1,A,UniProt,O60885,MKKGHHHHHHLVPRGSNPPPPETSNPNKPKRQTNQLQYLLRVVLKT...,141,polypeptide(L),16.767,NaN
4,5D15,1,"A, B",UniProt,V7LAR8,FQGAMGSRVVILFTDIEESTALNERIGDRAWVKLISSHDKLVSDLV...,177,polypeptide(L),19.587,NaN
...,...,...,...,...,...,...,...,...,...,...
28060,NaN,3,"C, D",UniProt,P35444,PSPCHEKADCILERDGSRS,19,polypeptide(L),2.12,NaN
28061,NaN,1,"A, H",NaN,NaN,QVQLQQSGAELAKPGASVNLSCKASGYTFTNYWVHWVKQRPGQGLE...,233,polypeptide(L),25.39,NaN
28062,6SET,1,A,UniProt,P00698,KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRN...,129,polypeptide(L),14.331,NaN
28063,6SEW,1,A,UniProt,P00698,KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRN...,129,polypeptide(L),14.331,NaN


I want to replace those sequences that have a NaN with the ID of the sequence that is above it

In [7]:
for index in fasta_df.index:
    #print(index)
    if pd.isnull(fasta_df.loc[index,"Entry ID"]):
        #print(fasta_df.loc[index,"Entry ID"])
        fasta_df.loc[index,"Entry ID"]=fasta_df.loc[index-1,"Entry ID"]
#fasta_df=fasta_df[fasta_df["Entry ID"].notna()]
fasta_df

,Entry ID,Entity ID,Chain ID,Database Name,Accession Code(s),Sequence,Chain Length,Entity Macromolecule Type,Molecular Weight (Entity),Unnamed: 9
0,5D13,1,"A, B, C, D",UniProt,P31016,GSPEFLGEEDIPREPRRIVIHRGSTGLGFNIVGGEDGEGIFISFIL...,119,polypeptide(L),12.738,NaN
1,5D13,2,"E, F, G, H",NaN,NaN,XKKETEV,7,polypeptide(L),0.939,NaN
2,5COQ,1,"A, B, C, D",UniProt,P9WGR1,MGSSHHHHHHSSGLVPRGSHMTGLLDGKRILVSGIITDSSIAFHIA...,289,polypeptide(L),30.698,NaN
3,5CP5,1,A,UniProt,O60885,MKKGHHHHHHLVPRGSNPPPPETSNPNKPKRQTNQLQYLLRVVLKT...,141,polypeptide(L),16.767,NaN
4,5D15,1,"A, B",UniProt,V7LAR8,FQGAMGSRVVILFTDIEESTALNERIGDRAWVKLISSHDKLVSDLV...,177,polypeptide(L),19.587,NaN
...,...,...,...,...,...,...,...,...,...,...
28060,6SF6,3,"C, D",UniProt,P35444,PSPCHEKADCILERDGSRS,19,polypeptide(L),2.12,NaN
28061,6SF6,1,"A, H",NaN,NaN,QVQLQQSGAELAKPGASVNLSCKASGYTFTNYWVHWVKQRPGQGLE...,233,polypeptide(L),25.39,NaN
28062,6SET,1,A,UniProt,P00698,KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRN...,129,polypeptide(L),14.331,NaN
28063,6SEW,1,A,UniProt,P00698,KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRN...,129,polypeptide(L),14.331,NaN


Now I want to delete those sequences that have less than 50 and more than 300 aminoacids

In [8]:
#REMOVING FIRST THE SEQUENCES THAT ARE LESS THAN 50 AA LONG
#pd.to_numeric(fasta_df["Chain Length"],errors="coerce")
#fasta_df_length=fasta_df[pd.to_numeric(fasta_df["Chain Length"],errors="coerce")>50]
fasta_df=fasta_df[pd.to_numeric(fasta_df["Chain Length"],errors="coerce")>50]
#REMOVING THEN THE SEQUENCES THAT ARE MORE THAN 300 AA LONG
fasta_df=fasta_df[pd.to_numeric(fasta_df["Chain Length"],errors="coerce")<300]
fasta_df

,Entry ID,Entity ID,Chain ID,Database Name,Accession Code(s),Sequence,Chain Length,Entity Macromolecule Type,Molecular Weight (Entity),Unnamed: 9
0,5D13,1,"A, B, C, D",UniProt,P31016,GSPEFLGEEDIPREPRRIVIHRGSTGLGFNIVGGEDGEGIFISFIL...,119,polypeptide(L),12.738,NaN
2,5COQ,1,"A, B, C, D",UniProt,P9WGR1,MGSSHHHHHHSSGLVPRGSHMTGLLDGKRILVSGIITDSSIAFHIA...,289,polypeptide(L),30.698,NaN
3,5CP5,1,A,UniProt,O60885,MKKGHHHHHHLVPRGSNPPPPETSNPNKPKRQTNQLQYLLRVVLKT...,141,polypeptide(L),16.767,NaN
4,5D15,1,"A, B",UniProt,V7LAR8,FQGAMGSRVVILFTDIEESTALNERIGDRAWVKLISSHDKLVSDLV...,177,polypeptide(L),19.587,NaN
5,5COS,1,"A, B, C, D",UniProt,A0A084CH09,GAMGMNGQGATSIPGEVAEQAMHWHLELQEPAVSAATLAACMSWRQ...,86,polypeptide(L),9.515,NaN
...,...,...,...,...,...,...,...,...,...,...
28059,6SF6,2,"B, L",NaN,NaN,DVLMTQIPLSLPVSLGDQASISCRSSQNIVHSNGNTYLEWYLQKPG...,219,polypeptide(L),24.236,NaN
28061,6SF6,1,"A, H",NaN,NaN,QVQLQQSGAELAKPGASVNLSCKASGYTFTNYWVHWVKQRPGQGLE...,233,polypeptide(L),25.39,NaN
28062,6SET,1,A,UniProt,P00698,KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRN...,129,polypeptide(L),14.331,NaN
28063,6SEW,1,A,UniProt,P00698,KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRN...,129,polypeptide(L),14.331,NaN


I want now to remove all those sequences that have the X in their AA sequence

In [9]:
fasta_df=fasta_df[fasta_df["Sequence"].str.contains("X")==False]
fasta_df.reset_index(drop=True, inplace=True)
fasta_df

,Entry ID,Entity ID,Chain ID,Database Name,Accession Code(s),Sequence,Chain Length,Entity Macromolecule Type,Molecular Weight (Entity),Unnamed: 9
0,5D13,1,"A, B, C, D",UniProt,P31016,GSPEFLGEEDIPREPRRIVIHRGSTGLGFNIVGGEDGEGIFISFIL...,119,polypeptide(L),12.738,NaN
1,5COQ,1,"A, B, C, D",UniProt,P9WGR1,MGSSHHHHHHSSGLVPRGSHMTGLLDGKRILVSGIITDSSIAFHIA...,289,polypeptide(L),30.698,NaN
2,5CP5,1,A,UniProt,O60885,MKKGHHHHHHLVPRGSNPPPPETSNPNKPKRQTNQLQYLLRVVLKT...,141,polypeptide(L),16.767,NaN
3,5D15,1,"A, B",UniProt,V7LAR8,FQGAMGSRVVILFTDIEESTALNERIGDRAWVKLISSHDKLVSDLV...,177,polypeptide(L),19.587,NaN
4,5COS,1,"A, B, C, D",UniProt,A0A084CH09,GAMGMNGQGATSIPGEVAEQAMHWHLELQEPAVSAATLAACMSWRQ...,86,polypeptide(L),9.515,NaN
...,...,...,...,...,...,...,...,...,...,...
23868,6SF6,2,"B, L",NaN,NaN,DVLMTQIPLSLPVSLGDQASISCRSSQNIVHSNGNTYLEWYLQKPG...,219,polypeptide(L),24.236,NaN
23869,6SF6,1,"A, H",NaN,NaN,QVQLQQSGAELAKPGASVNLSCKASGYTFTNYWVHWVKQRPGQGLE...,233,polypeptide(L),25.39,NaN
23870,6SET,1,A,UniProt,P00698,KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRN...,129,polypeptide(L),14.331,NaN
23871,6SEW,1,A,UniProt,P00698,KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRN...,129,polypeptide(L),14.331,NaN


## Clusterization procedure

Now I have to perform the clusterization procedure, but I have to convert the DataFrame in a multifasta file

In [109]:
!touch > fasta.fasta

touch: missing file operand
Try 'touch --help' for more information.


In [110]:
fasta_file=open("fasta.fasta","a")
for index in fasta_df.index:
    PDB_ID=">"+fasta_df.loc[index,"Entry ID"]+"_"+fasta_df.loc[index,"Chain ID"].split(",")[0]
    Sequence=fasta_df.loc[index,"Sequence"]
    print(PDB_ID,file=fasta_file)
    #print("\n",file=fasta_file)
    print(Sequence,file=fasta_file)    
#fasta_file.write(fasta_df.to_string(fasta_df["Sequence"]))
fasta_file.close()

In [6]:
!grep ">" fasta.fasta | wc

  23873   23873  191260


Now I have to run the mmseqs clusterization package

In [112]:
!mmseqs easy-cluster fasta.fasta Clustering_res tmp --min-seq-id 0.3 -c 0.5 --cluster-mode 0 

easy-cluster fasta.fasta Clustering_res tmp --min-seq-id 0.3 -c 0.5 --cluster-mode 0 

MMseqs Version:                     	12.113e3
Substitution matrix                 	nucl:nucleotide.out,aa:blosum62.out
Seed substitution matrix            	nucl:nucleotide.out,aa:VTML80.out
Sensitivity                         	4
k-mer length                        	0
k-score                             	2147483647
Alphabet size                       	nucl:5,aa:21
Max sequence length                 	65535
Max results per query               	20
Split database                      	0
Split mode                          	2
Split memory limit                  	0
Coverage threshold                  	0.5
Coverage mode                       	0
Compositional bias                  	1
Diagonal scoring                    	true
Exact k-mer matching                	0
Mask residues                       	1
Mask lower case residues            	0
Minimum diagonal score              	15
Include identical seq. id.   

In [10]:
!grep ">" Clustering_res_rep_seq.fasta | wc -l

4086


The file in which we are interested is the "Clustering_res_rep_seq.fasta"

## Removing redundancy between sets

I have now to open the "Tot_sequence_JPred4.fasta"

And now that I have my Blind set of representative protein sequences and the JPred4 fasta I can remove the redundacy between these two sets by blasting them one against the other. And I want to retain only those proteins that have low sequence identity.  

Now I have to make the Tot_sequence_JPred4.fasta as the database for blasting against it the Blindset ("Clustering_res_rep_seq.fasta")

In [114]:
!makeblastdb -in Tot_sequence_JPred4.fasta -dbtype prot



Building a new DB, current time: 09/23/2020 01:03:10
New DB name:   /mnt/EA7C9C107C9BD5A3/User/BIOINFORMATICA.BOLOGNA/LAB_2/Project/Project2/Tot_sequence_JPred4.fasta
New DB title:  Tot_sequence_JPred4.fasta
Sequence type: Protein
Deleted existing Protein BLAST database named /mnt/EA7C9C107C9BD5A3/User/BIOINFORMATICA.BOLOGNA/LAB_2/Project/Project2/Tot_sequence_JPred4.fasta
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 1348 sequences in 0.142454 seconds.


In [115]:
!blastp -query Clustering_res_rep_seq.fasta -db Tot_sequence_JPred4.fasta -evalue 0.01 -out hits_blast.tab -outfmt 6

The generated file is the "hits_blast.tab" 

Now I have to remove the redundancy between the two datasets by removing all the sequences that have more than 30% of sequence identity

In [11]:
hits=open("hits_blast.tab")
list_ID=[]
for line in hits:
    list_line=[]
    list_line=line.split("\t")
    if float(list_line[2])>30.00:
        if list_line[0] not in list_ID:
             list_ID.append(list_line[0])
print(len(list_ID))
#print(list_ID)

555


In the list_ID there are all the fasta ID that we want to remove from the 4086 proteinf in the Clustering_res_rep_seq.fasta file

In [12]:
rep=open("Clustering_res_rep_seq.fasta","r")
rep_dict={}
rep_ID=[]
for line in rep:
    if line.startswith(">"):
        #print(line[1:7])
        line=line.rstrip()[1:7]
        #print(line)
        if line not in list_ID:
            keys=rep_dict.keys()
            if line not in rep_ID:
                rep_ID.append(line)
                if line not in keys:
                    rep_dict[line]=rep.readline().rstrip()    
rep_df=pd.DataFrame.from_dict(rep_dict,orient="index") #IMPORTANT SYNTAX!!
rep_df.reset_index(inplace=True) #IMPORTANT SYNTAX!!!
rep_df.columns=["ID","Sequence"] #IMPORTANT SYNTAX!!!
print(len(rep_ID))
rep_df

3532


,ID,Sequence
0,4XXP_A,MAHHHHHHASPAPPPAAPSGQSGTYLFQDEFDGPAGSAPDASKWAV...
1,4XXF_A,GGLAALTKPPTFATVEAERAWLKERLVAAIRIFANEGFDHTVAGHL...
2,4XXI_A,MLAVATITQAEQQDRFLGRGELDELASYFASGAKRLEIAQLLTENS...
3,4Y9I_A,GVHVLDRPIVLFTTTGAKSGKKRYVPLMRVEENGKYAMVASKGGDP...
4,4XXB_A,MAQDQGEKENPMRELRIRKLCLNICVGESGDRLTRAAKVLEQLTGQ...
...,...,...
3527,6M89_A,GSFTSHQVAGHMYGKDKVGILQHPDGTVLKQLQPPPRGPRELEFYN...
3528,6LW3_A,GPLGSGRPKKMTLILGIDPGSRITGFGVVRETARGCEYVASGCIRT...
3529,6M96_B,MNLSLILELVRQEIKNRYADTVLGIWWAFLWPILLVLIYTLIFSHL...
3530,6M9K_D,ITPVNDETMQEINTLLIALDKTWDDDLLPLCSQIFRRDIRASSELT...


In the rep_ID there are all the ID of the representative set (our blind set) from which we are now going to randomly extract 150 sequence ID that we want to download to PDB

## Donwload PDB files

I will now randomly extract only 150 PDB ID from the rep_ID list that I will then download from PDB

In [75]:
#import random
def randomize(rep_ID):
    randomlist=[]
    random.seed(22092020) # RANDOM SEED!!!!!!!!!!!!!
    while len(randomlist)!=150:
        n=random.randint(0,len(rep_ID)-1)
        if n not in randomlist:
            randomlist.append(n)
    return(randomlist)

randomlist=randomize(rep_ID)

def random_ID(rep_ID,randomlist):
    PDB_ID=[]
    for num in randomlist:
        PDB_ID.append(rep_ID[int(num)])
    return(PDB_ID)

PDB_ID=random_ID(rep_ID,randomlist)

print(len(PDB_ID))
#print(PDB_ID)

150
['4XXT_A', '5XDH_A', '5DJT_B', '4ZV9_A', '5ZZ6_A', '5Z48_A', '6NMW_A', '5EZU_A', '4YSL_A', '5IYS_A', '5HX4_A', '5GKB_A', '5VKD_A', '6KSY_A', '6EWH_A', '6IN9_C', '5YGE_A', '4XU4_A', '6K2H_A', '6ELU_A', '5N2I_A', '6A45_A', '5GKK_A', '6GJE_B', '6HRR_A', '4YH1_A', '6JCC_A', '4YHV_A', '5VHT_A', '6QJA_A', '4YMR_A', '6SPZ_A', '6O2V_A', '5VJT_A', '6J2Z_A', '6SSH_A', '6ESK_A', '5VYM_A', '6BDC_A', '5OHF_A', '4XXF_A', '6DN5_A', '5HI3_A', '5ME7_A', '5FCU_G', '5FSB_A', '6I6J_A', '6GOS_A', '5DY9_A', '5X6L_A', '6PUP_A', '4YUE_C', '6W2L_B', '5E5M_A', '5FV5_A', '6IN8_A', '6OM5_A', '5Z4Y_A', '5OIY_A', '5FEB_A', '5NFD_A', '6DTC_A', '6OF0_A', '5JO8_A', '6FDD_A', '5IQJ_A', '4XZX_A', '6NHL_A', '4YK1_A', '5YHH_A', '4YAP_A', '6T84_A', '6HAT_A', '6QO1_A', '6MD3_A', '6TUM_A', '6YMU_B', '6W04_A', '5H5E_A', '6KN5_A', '5LXO_A', '6FLM_A', '5X7L_A', '4YBH_B', '6AHT_A', '5FAI_A', '5F7T_E', '6I8B_B', '5B4B_A', '6L4P_B', '5Y21_A', '6H4K_A', '6F3D_A', '5DQS_A', '5H9K_A', '7C2G_G', '4ZC4_A', '5JR5_A', '6FPK_A', '6BP6

#### Making new directories !!!

In [52]:
!mkdir 150_PDB_structures

In [169]:
!mkdir rep_PDB

Now I want to download from PDB all the PDB structures of the representative sequences (3532)

In [176]:
#from Bio.PDB.PDBList import *
PDB_ID_NOchain=[]
for ID in rep_ID:
    if ID[0:4] not in PDB_ID_NOchain:
        PDB_ID_NOchain.append(ID[0:4])
#Now I can downoad the PDB files
pdbl=PDBList()
pdbl.download_pdb_files(PDB_ID_NOchain,pdir="rep_PDB/",file_format="pdb")

KeyboardInterrupt: 

## DSSP Generation

Now that I've downloaded all the PDB stuctures I want to perform the DSSP on them.

I want now to save in the rep_ID only those files for which I have also the BDP downloaded in the folder rep_PDB (that contains 3336 PDB files).
Because some of them are too big and I could not download them from PDB

In [287]:
list_rep_PDB=[]
rep_PDB=[]
for filename in os.listdir("rep_PDB"):
    list_rep_PDB.append(filename[3:7].upper())
#print(len(list_rep_PDB))
for ID in rep_ID:
    if ID[0:4] in list_rep_PDB:
        rep_PDB.append(ID)
print(len(rep_PDB))

3518


In [3]:
#dssp_df=pd.DataFrame(columns=["ID","Chain","DSSP","Primary Structure","len"])
def convert(el):
     if el == "H" or el =="G" or el =="I":
        return "H"
     elif el =="B" or el =="E":
        return "E"
     elif el=="T" or el =="S" or el =="-":
        return "C"

def sub_lower(aa):
    if aa.islower():
        return "C"
    else:
        return aa

def random(rep_PDB):
    import random
    ran=random.sample(rep_PDB,1)[0]
    return(ran)

ID=random(rep_PDB)
ID_list=[]
dssp_df2=pd.DataFrame(columns=["ID","Chain","DSSP","Primary Structure"])
while len(dssp_df2)!=150: #Specify the number of sequeces that you want in your dataframe
    if ID not in ID_list:
        ID_list.append(ID)
        prim_str=""
        sec_str=""
        #print(ID)
        dssp_tuple=dssp_dict_from_pdb_file("rep_PDB/pdb"+ID[0:4].lower()+".ent",DSSP="mkdssp")
        dssp_dict=dssp_tuple[0]
        index=1
        for line in dssp_dict:
            if line[0]==ID[5]:
                if index==1:
                    temp_line=line[1][1]
                    prim_str=prim_str+sub_lower(dssp_dict[line][0])
                    sec_str=sec_str+convert(dssp_dict[line][1])
                    index=index+1
                else:
                    actual_line=line[1][1]
                    if actual_line-1==temp_line:
                        prim_str=prim_str+sub_lower(dssp_dict[line][0])
                        sec_str=sec_str+convert(dssp_dict[line][1])
                    else:
                        prim_str=prim_str+"error" #X
                        sec_str=sec_str+"C"
                    temp_line=line[1][1]
        if "error" not in prim_str and len(prim_str)>50:
            dssp_df2=dssp_df2.append({"ID":ID[0:4],"Chain":ID[5],"DSSP":sec_str,"Primary Structure":prim_str,"len":int(len(sec_str)) },ignore_index=True)
    else:
        ID=random(rep_PDB)
        
dssp_df2

NameError: name 'rep_PDB' is not defined

In [318]:
dssp_final=dssp_df2
dssp_final

,ID,Chain,DSSP,Primary Structure,len
0,5L8X,A,CCCCCCCCCCCCCCCCCCCCEECCCCCCEEEECCCCCCCHHHHHHC...,SHMANKREPAPGWPIVSGEYVVGNPESCVGVVTLGSHGLEQACIDA...,172.0
1,5WOQ,A,CCCHHHHHHHHHHHHHHCCCCCHHHHHHHHCCCHHHHHHHHCCCCC...,TALLREVIGDVLRNARTDQGRTLREVSDAARVSLGYLSEVERGRKE...,80.0
2,4ZPU,A,CCHHHHHHHHHCHHHHHHHHHHHHHHCCCCCCCCCCCEEEEEECCC...,MPPSLRKAVAAAIGGGAIAIASVLITGPSGNDGLEGVSYIPYKDIV...,165.0
3,6GMR,B,CEEEEEEEEECEEEEEEEECCCEHHHHHHHHHHHHCCCHHHEEEEE...,MDVFLMIRRHKTTIFTDAKESSTVFELKRIVEGILKRPPDEQRLYK...,105.0
4,5WSD,A,CCCCEEEEEHHHCCCEEEEECCEEEEEEEEEECCCCCCCEEEEEEE...,GPSGMILKRAYDVTPQKISTDKVRGVRKRVLIGLKDAPNFVMRLFT...,118.0
...,...,...,...,...,...
145,6JL3,A,CCEEEEEEECCCEEEEEEECCCCEHHHHHHHHHCCCCCCHHHEEEE...,MVINVSFKVTGGKEFTVAIEPDITVLDLKKICAEHVDIPVEAQRII...,74.0
146,6DRE,B,CHHHHHHHHHHHHHHHHHHHHHCCCCHHHHHHHHHHCCHHHHHHHH...,EQAKVWTQTARANAEKNNAQLSTLLTDDQIGAIYGYTTNEGYTALN...,170.0
147,5K31,A,CCCHHHHHHHHHHHHHHHHHHHHCCCCCCCCCECCHHHHHHHCCCC...,VRDRDLEVDTTLKSLSQQIENIRSPEGSRKNPARTCRDLKMCHSDW...,241.0
148,6S8X,A,CHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH...,TAGYRSLTYEEVLQELVKHKELLRRKDTHIRELEDYIDNLLVRVME...,53.0


In [332]:
dssp_final

,ID,Chain,DSSP,Primary Structure,len
0,5L8X,A,CCCCCCCCCCCCCCCCCCCCEECCCCCCEEEECCCCCCCHHHHHHC...,SHMANKREPAPGWPIVSGEYVVGNPESCVGVVTLGSHGLEQACIDA...,172.0
1,5WOQ,A,CCCHHHHHHHHHHHHHHCCCCCHHHHHHHHCCCHHHHHHHHCCCCC...,TALLREVIGDVLRNARTDQGRTLREVSDAARVSLGYLSEVERGRKE...,80.0
2,4ZPU,A,CCHHHHHHHHHCHHHHHHHHHHHHHHCCCCCCCCCCCEEEEEECCC...,MPPSLRKAVAAAIGGGAIAIASVLITGPSGNDGLEGVSYIPYKDIV...,165.0
3,6GMR,B,CEEEEEEEEECEEEEEEEECCCEHHHHHHHHHHHHCCCHHHEEEEE...,MDVFLMIRRHKTTIFTDAKESSTVFELKRIVEGILKRPPDEQRLYK...,105.0
4,5WSD,A,CCCCEEEEEHHHCCCEEEEECCEEEEEEEEEECCCCCCCEEEEEEE...,GPSGMILKRAYDVTPQKISTDKVRGVRKRVLIGLKDAPNFVMRLFT...,118.0
...,...,...,...,...,...
145,6JL3,A,CCEEEEEEECCCEEEEEEECCCCEHHHHHHHHHCCCCCCHHHEEEE...,MVINVSFKVTGGKEFTVAIEPDITVLDLKKICAEHVDIPVEAQRII...,74.0
146,6DRE,B,CHHHHHHHHHHHHHHHHHHHHHCCCCHHHHHHHHHHCCHHHHHHHH...,EQAKVWTQTARANAEKNNAQLSTLLTDDQIGAIYGYTTNEGYTALN...,170.0
147,5K31,A,CCCHHHHHHHHHHHHHHHHHHHHCCCCCCCCCECCHHHHHHHCCCC...,VRDRDLEVDTTLKSLSQQIENIRSPEGSRKNPARTCRDLKMCHSDW...,241.0
148,6S8X,A,CHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH...,TAGYRSLTYEEVLQELVKHKELLRRKDTHIRELEDYIDNLLVRVME...,53.0


I want to check if the percentage of the secondary structures (C-coil, E-beta sheet, H-helix) in the DSSP are more or less balanced.

In [4]:
dict={}
tot=0
for dssp in os.listdir("Blindset_DSSP"):
    dssp=open("Blindset_DSSP/"+dssp)
    for line in dssp:
        if line.startswith(">"):
            dssp=dssp.readline().rstrip()
            for el in dssp:
                key=dict.keys()
                if el in key:
                    dict[el]+=1
                else:
                    dict[el]=1
for key in dict:
    tot=tot+dict[key]
for key in dict:
    print("Percentage of " , key,": ", (dict[key]/tot)*100,"%")

Percentage of  C :  36.504352110362944 %
Percentage of  H :  40.396616850057484 %
Percentage of  E :  23.09903103957957 %


I can easily make a cake graph to put in the report

## Saving the DSSP and FASTA files

Now I want to save all the 150 DSSP and fasta in separate files in separate folders

In [339]:
!mkdir Blindset_Fasta

In [340]:
!mkdir Blindset_DSSP

In [341]:
for index in dssp_final.index:
    ID=">"+dssp_final.loc[index,"ID"]+"_"+dssp_final.loc[index,"Chain"]
    fasta=dssp_final.loc[index,"Primary Structure"]
    file=open("Blindset_Fasta/"+ID[1:]+".fasta","w+")
    file.write(ID)
    file.write("\n")
    file.write(fasta)

In [342]:
file.close()

In [343]:
for index in dssp_final.index:
    ID=">"+dssp_final.loc[index,"ID"]+"_"+dssp_final.loc[index,"Chain"]
    DSSP=dssp_final.loc[index,"DSSP"]
    file=open("Blindset_DSSP/"+ID[1:]+".dssp","w+")
    file.write(ID)
    file.write("\n")
    file.write(DSSP)

In [344]:
file.close()

## Running PSI-BLAST

First I have to convert the "uniprot_sprot.fasta" file in a database so that I can then blast against it the fasta sequences of the Blind set (test set)/JPred4 (training set) sets. 

In [363]:
!makeblastdb -in uniprot_sprot.fasta -dbtype prot



Building a new DB, current time: 09/25/2020 19:07:29
New DB name:   /mnt/EA7C9C107C9BD5A3/User/BIOINFORMATICA.BOLOGNA/LAB_2/Project/Project2/uniprot_sprot.fasta
New DB title:  uniprot_sprot.fasta
Sequence type: Protein
Deleted existing Protein BLAST database named /mnt/EA7C9C107C9BD5A3/User/BIOINFORMATICA.BOLOGNA/LAB_2/Project/Project2/uniprot_sprot.fasta
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 563082 sequences in 34.8935 seconds.


Now I can run psi-blast.
Documentation: https://biopython.org/docs/1.75/api/Bio.Blast.Applications.html

In [ ]:
!mkdir Psiblast_alns

In [ ]:
!mkdir Psiblast_pssm

In [369]:
#from Bio.Blast.Applications import NcbipsiblastCommandline
for fasta_file in os.listdir("Blindset_Fasta/"):
    cline=NcbipsiblastCommandline(query="Blindset_Fasta/"+fasta_file,
                                 db='uniprot_sprot.fasta',
                                 evalue= 0.01,
                                 num_iterations=3,
                                 out="Psiblast_alns/"+fasta_file[0:6]+".alns.blast",
                                 out_ascii_pssm="Psiblast_pssm//"+fasta_file[0:6]+".pssm", #I could put another directory for the output
                                 num_descriptions=10000,
                                 num_alignments=10000,
                                 num_threads=4)
    cline()

In [395]:
!mkdir Psiblast_alns_JPred

In [396]:
!mkdir Psiblast_pssm_JPred

In [397]:
for fasta_file in os.listdir("fasta_Jpred/"):
    base_name=os.path.basename(fasta_file)
    name=os.path.splitext(base_name)[0]
    cline=NcbipsiblastCommandline(query="fasta_Jpred/"+base_name,
                                 db='uniprot_sprot.fasta',
                                 evalue= 0.01,
                                 num_iterations=3,
                                 out="Psiblast_alns_JPred/"+name+".alns.blast",
                                 out_ascii_pssm="Psiblast_pssm_JPred/"+name+".pssm", #I could put another directory for the output
                                 num_descriptions=10000,
                                 num_alignments=10000,
                                 num_threads=4)
    cline()

## Extract the Profile matrices

Now I want to extract only the Profile matrices from the .pssm files and save the pssm matrix in the new folder. I have to normalize (by dividing by 100) all the value in the matrxi and delete all those those profile that have a matrix composide.

In [25]:
!mkdir Blindset_pssm

In [4]:
def pssm_fun(pssm):
    pssm_list=[]
    tot=0
    for line in pssm:
        if "Last" in line:
            colname=pssm.readline().split()[20:]
            for line in pssm:
                line_list=[]
                line_list=line.rstrip().split()[22:42]
                if line_list==[]:
                    break
                else:
                    for el in range(len(line_list)):
                        line_list[el]=int(line_list[el])/100
                        tot=tot+float(line_list[el])
                    pssm_list.append(line_list)
    pssm_df=pd.DataFrame(pssm_list,columns=colname)
    pssm_df.index=pssm_df.index+1
    return(pssm_df,tot)

count=0
for pssm in os.listdir("Psiblast_pssm"):
    if pssm.endswith(".pssm"):
        pssm_file=open("Psiblast_pssm/"+pssm)
        pssm_name=pssm.split(".")
        pssm_name=".".join(pssm_name[:-1])
        pssm_df,tot=pssm_fun(pssm_file)
        #pssm_df
        if tot==0:
            count+=1
            print("This ", count,"° PSSM has a matrxi of 0s only: " ,pssm_name)
        else:
            pssm_df.to_csv("Blindset_pssm/"+pssm_name+".pssm",sep="\t")

This  1  PSSM has a matrxi of 0s only:  5ABV_B
This  2  PSSM has a matrxi of 0s only:  5VDB_A
This  3  PSSM has a matrxi of 0s only:  5MU3_B
This  4  PSSM has a matrxi of 0s only:  6NNW_A
This  5  PSSM has a matrxi of 0s only:  6B4H_B


In [27]:
!mkdir JPred_pssm

In [5]:
def pssm_fun(pssm):
    pssm_list=[]
    tot=0
    for line in pssm:
        if "Last" in line:
            colname=pssm.readline().split()[20:]
            for line in pssm:
                line_list=[]
                line_list=line.rstrip().split()[22:42]
                if line_list==[]:
                    break
                else:
                    for el in range(len(line_list)):
                        line_list[el]=int(line_list[el])/100
                        tot=tot+float(line_list[el])
                    pssm_list.append(line_list)
    pssm_df=pd.DataFrame(pssm_list,columns=colname)
    pssm_df.index=pssm_df.index+1
    return(pssm_df,tot)

count=0
for pssm in os.listdir("Psiblast_pssm_JPred"):
    if pssm.endswith(".pssm"):
        pssm_file=open("Psiblast_pssm_JPred/"+pssm)
        pssm_name=pssm.split(".")
        pssm_name=".".join(pssm_name[:-1])
        pssm_df,tot=pssm_fun(pssm_file) #function recall
        #print(pssm_df)
        if tot==0:
            count+=1
            print("This ", count," PSSM has a matrxi of 0s only: " ,pssm_name)
        else:
            pssm_df.to_csv("JPred_pssm/"+pssm_name+".pssm",sep="\t")
#pssm_df

This  1  PSSM has a matrxi of 0s only:  d1v74b_
This  2  PSSM has a matrxi of 0s only:  d1vk5a_
This  3  PSSM has a matrxi of 0s only:  d1w53a_
This  4  PSSM has a matrxi of 0s only:  d3h7ia2
This  5  PSSM has a matrxi of 0s only:  d2ebfx1
This  6  PSSM has a matrxi of 0s only:  d2enda_
This  7  PSSM has a matrxi of 0s only:  d2erla_
This  8  PSSM has a matrxi of 0s only:  d2euca1
This  9  PSSM has a matrxi of 0s only:  d2f2ha1
This  10  PSSM has a matrxi of 0s only:  d2gsva1
This  11  PSSM has a matrxi of 0s only:  d1ryla_
This  12  PSSM has a matrxi of 0s only:  d1yoza1
This  13  PSSM has a matrxi of 0s only:  d1yu0a1
This  14  PSSM has a matrxi of 0s only:  d1s7za_
This  15  PSSM has a matrxi of 0s only:  d1seda_
This  16  PSSM has a matrxi of 0s only:  d1sf9a_
This  17  PSSM has a matrxi of 0s only:  d1pv5a_
This  18  PSSM has a matrxi of 0s only:  d1e7la1
This  19  PSSM has a matrxi of 0s only:  d1el6a_
This  20  PSSM has a matrxi of 0s only:  d1ezga_
This  21  PSSM has a matrxi o

## Develop the Gor-training program

The **GOR method** is based on probability parameters derived from empirical studies of known protein tertiary structures solved by X-ray crystallography. However, unlike the Chou-Fasman, the Gor method takes into account not only the propensity of individual amino acids to form particular secondary structures, but also the conditional probability of the amino acid to form a secondary structure given that its immediate neighbours have already formed that structure. The methods is essentially **Bayesian** in its analysis. 

I have to pick each profile matrix that we have generated and extracted from PsiBlast and generate the gor matrix

**Put here the path of the folder with the profiles and the secondary structure of the sequences for which you want to do the training :**

In [29]:
pssm_path="JPred_pssm/" #Profile path
dssp_path="JPred_dssp/" #Secondary structure path

In [30]:
%%time 
aa=['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']
secstr=["H","E","C","tot"]
window=17
gor_list_mat=[]
for el in secstr:
    gor_list_mat.append([el,np.zeros(((window),len(aa)),float)])   
    
def extracter(dssp,pssm):
    for line in dssp:
        if line.startswith(">"):
            sec_str1=""
            sec_str=dssp.readline().rstrip()
            for el in sec_str: 
                if el=="-":
                    sec_str1=sec_str1+"C"
                else:
                    sec_str1=sec_str1+el 
    for line in pssm:
        if "A" not in line:
            line=line.rstrip()
            line=line.split()[1:]
            pssm_list.append(line)
    return(sec_str1,pssm_list)

def window_pssm(pssm_list,sec_str,window,el):
        window_array=pssm_list[el-(window//2):el+window//2+1]
        return(window_array,sec_str[el-window//2])

def add_window_spaces(pssm_list,window):
    for i in range(window//2):
        pssm_list=[[0.0 for i in range(20)]]+pssm_list
        pssm_list.append([0.0 for i in range(20)])
    pssm_array=np.array([np.array(list) for list in pssm_list])
    pssm_list_float=pssm_array.astype(np.float)
    return(pssm_list_float)

def fill_gor_mat(window_array,el_sec_str,gor_list_mat):
    for list in range(len(gor_list_mat)):
        if gor_list_mat[list][0]==el_sec_str:
            gor_list_mat[list][1]+=window_array
    gor_list_mat[3][1]+=window_array
    return(gor_list_mat)

def gor_df_fun(tot_gor_mat,secstr,aa,window):
    index=[]
    gor_df1=pd.DataFrame(tot_gor_mat)
    gor_df1.columns=aa
    for el in secstr:
        for i in range(window):
            index.append("R"+str(i-(window//2))+"_"+el)
    gor_df1.index=index
    return(gor_df1)

def normalization(gor_list_mat):
    summation=[]
    summation=np.sum(gor_list_mat[3][1],axis=1)
    for array in gor_list_mat:
        for sub in range(len(array[1])):
            array[1][sub]=array[1][sub]/summation[sub]
    return(gor_list_mat)

prob_sec_str={}
#print(prob_sec_str[1])
for pssm_file in os.listdir(pssm_path):
    #print(pssm_file)
    pssm_list=[]
    pssm_file=pssm_file.split(".")
    pssm_file=".".join(pssm_file[:-1])
    pssm=open(pssm_path+pssm_file+".pssm")
    dssp=open(dssp_path+pssm_file+".dssp")
    sec_str,pssm_list=extracter(dssp,pssm) # function recall
    pssm_array=add_window_spaces(pssm_list,window) #function recall
    for el in range(window//2,len(sec_str)+window//2):
        window_array,el_sec_str=window_pssm(pssm_array,sec_str,window,el) #function recall
        gor_list_mat=fill_gor_mat(window_array,el_sec_str,gor_list_mat) #function recall
        keys=prob_sec_str.keys()
        if el_sec_str in keys:
            prob_sec_str[el_sec_str]+=1
        else: 
            prob_sec_str[el_sec_str]=1
#print(gor_list_mat)
gor_list_mat=normalization(gor_list_mat) #function recall
tot_gor_mat=np.concatenate((gor_list_mat[0][1],gor_list_mat[1][1],gor_list_mat[2][1],gor_list_mat[3][1]),axis=0)
gor_df=gor_df_fun(tot_gor_mat,secstr,aa,window) #fucntion recall
print("Number of the secondary structures: ",prob_sec_str)
gor_df

Number of the secondary structures:  {'C': 84297, 'H': 70534, 'E': 43972}
CPU times: user 5.84 s, sys: 181 ms, total: 6.02 s
Wall time: 9.37 s


,A,R,N,D,C,Q,E,G,H,I,L,K,M,F,P,S,T,W,Y,V
R-8_H,0.031335,0.018633,0.014761,0.021476,0.004575,0.014522,0.026603,0.023597,0.008305,0.020903,0.035207,0.021388,0.008456,0.013840,0.015724,0.022866,0.019575,0.004304,0.011312,0.023634
R-7_H,0.032013,0.018729,0.014715,0.021577,0.004574,0.014702,0.027173,0.023042,0.008143,0.020807,0.035373,0.021351,0.008483,0.013792,0.015211,0.022843,0.019518,0.004285,0.011227,0.023504
R-6_H,0.032711,0.018709,0.014504,0.021578,0.004625,0.014919,0.027531,0.022268,0.008049,0.021052,0.036033,0.021327,0.008552,0.013814,0.014686,0.022586,0.019409,0.004293,0.011017,0.023378
R-5_H,0.033496,0.018969,0.014333,0.021467,0.004577,0.015187,0.028119,0.021410,0.007920,0.021205,0.036702,0.021536,0.008618,0.013613,0.014068,0.022236,0.019130,0.004277,0.010902,0.023181
R-4_H,0.034159,0.019174,0.014201,0.021362,0.004424,0.015591,0.028883,0.020069,0.007799,0.021404,0.037283,0.021816,0.008687,0.013470,0.013333,0.021942,0.018862,0.004315,0.010845,0.023028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
R4_tot,0.081727,0.051768,0.041783,0.057616,0.014798,0.038320,0.067987,0.071549,0.022834,0.060092,0.095151,0.059887,0.020857,0.039542,0.043656,0.061267,0.053969,0.013180,0.033308,0.070707
R5_tot,0.081802,0.051783,0.041794,0.057669,0.014804,0.038295,0.067940,0.071696,0.022848,0.060033,0.095121,0.059897,0.020861,0.039552,0.043544,0.061239,0.053944,0.013191,0.033342,0.070646
R6_tot,0.081889,0.051763,0.041819,0.057704,0.014796,0.038281,0.067898,0.071829,0.022907,0.059915,0.095113,0.059874,0.020858,0.039556,0.043509,0.061282,0.053904,0.013197,0.033330,0.070576
R7_tot,0.081975,0.051712,0.041873,0.057743,0.014778,0.038265,0.067881,0.071950,0.022905,0.059925,0.095047,0.059776,0.020869,0.039515,0.043529,0.061289,0.053884,0.013182,0.033347,0.070555


If you want to save and print the profile to a file:

In [26]:
gor_df.to_csv("Gor_matrix.csv",sep="\t")

## Prediction phase

Now I have to apply the formula to calculate the **Information function**:

$I(S;R)= \log\frac{P(R,S)}{P(S)P(R)}$

Where:
- Each cell of the Gor matrix that we have generated is the $P(S;R)$
- $P(S)$ (*secondary structure*) is the one that I have caluclated in the code cell above (*but can also be calculate by summing the row corresponding on the position 0 of each E,H and C submatrices*).
- $P(R)$ (*probability of the residue*) is computed by summing the columns corresponding to each of the 20 residues that we have count at the position R_0.

I will now calculate the **marginal probaility** $P(S)$ of the secondary structures (C,H, and E) from the count that I made in the training phase.  

In [31]:
tot_sec_str=0
for el in prob_sec_str:
    tot_sec_str+=prob_sec_str[el]
for el in prob_sec_str:
    prob_sec_str[el]=prob_sec_str[el]/tot_sec_str
print(prob_sec_str)

{'C': 0.4240227763162528, 'H': 0.3547934387308039, 'E': 0.22118378495294336}


I will now extract the $P(R_k)$ from the gor matrices (*gor_list_mat*)

In [28]:
info_list_fun=gor_list_mat

In [29]:
for mat in info_list_fun:
    if mat[0]=="tot":
        tot_info_fun=mat[1]
#print(tot_info_fun)

Now that I have both the **marginal probabilities** ($P(R)$ and $P(S)$) and the **join probability** $P(R,S)$ from the trained gor model I can compute the **information function** matrix.

In [30]:
for ar in (info_list_fun[:-1]):
    for vec in range(len(ar[1])):
        for el in range(len(ar[1][vec])):
            P_ss=prob_sec_str[ar[0]]
            P_join=ar[1][vec][el]
            P_res=tot_info_fun[vec][el]
            ar[1][vec][el]=math.log2(P_join/(P_ss*P_res))

In [9]:
print(info_list_fun)

[['H', array([[ 1.12899949e-01,  3.60889194e-02, -1.19113808e-02,
         6.06961512e-02, -1.87352378e-01,  1.01648860e-01,
         1.50661258e-01, -1.11056672e-01,  3.25065448e-02,
        -2.46135331e-02,  7.39781451e-02,  2.31126941e-02,
         1.33951036e-01, -1.29835795e-02,  1.29945522e-02,
         5.70814166e-02,  1.52666965e-02, -1.09863415e-01,
        -6.11998612e-02, -8.38485051e-02],
       [ 1.44021032e-01,  4.16031656e-02, -1.61565469e-02,
         6.68855721e-02, -1.89938072e-01,  1.18013262e-01,
         1.79106789e-01, -1.43186399e-01,  4.71047247e-03,
        -3.13506966e-02,  8.00712066e-02,  1.88369036e-02,
         1.38697016e-01, -1.64531584e-02, -3.39645220e-02,
         5.64504447e-02,  1.19516394e-02, -1.14363722e-01,
        -7.19618367e-02, -9.03737781e-02],
       [ 1.76082080e-01,  3.76835838e-02, -3.53023167e-02,
         6.94593699e-02, -1.77307843e-01,  1.39754536e-01,
         1.96575874e-01, -1.90597242e-01, -1.04675697e-02,
        -1.58843040e-0

To save the information function

Now that I have calculated the information function for each cell of the gor model, I can perform the prediction of the secondary structure of a new sequence (*from the blind set*) from the profile that I have generated using *psiblast*. 

The predicted conformation of a residue R at position j is the one having the **highest value of the window-based information function**:

$S* = argmax_s I(S;R_{-d},...,R_d)= argmax_s\sum_{k=-d}^d I(S;R_k)$

For the prediction phase indeed also the profile of the sequences (*blind-set*) for which we want to predict the secondary structure is taken into consideration:


$PW$ (Profile Window) $= \begin{matrix} -1 & A:0.2 & I:0.8 \\ 0 & D:0.9 & N:0.1 \\ +1 & E:0.3 & P:0.7 \end{matrix}$  

$I(H;PW)= \sum_{k=-1}^1 \sum_R PW_{k,R} \times I(H;R_k) = 0.2 \times I(H;A_{-1}) + 0.8 \times I(H;I_{-1}) + 0.9 \times I(H;D_0) + 0.1 \times I(H;N_0) + 0.3 \times I(H;E_1) + 0.7 \times I(H;P_1)$

#### Prediction phase

In [31]:
prof_path="Blindset_pssm/" #Path of the sequence profile for which we wanna predict the secondary structure

In [32]:
#import math
#window=17
def reading_profile(profile):
    prof_list=[]
    long=0
    for line in profile:
        if "A" not in line:
            #prof_line=[]
            line=line.rstrip()
            array=line.split()[1:]
            prof_list.append(array)
            long+=1
    return(prof_list,long)

def add_window_spaces(prof_list,window):
    for i in range(window//2):
        prof_list=[[0.0 for i in range(20)]]+prof_list
        prof_list.append([0.0 for i in range(20)])
    prof_list=np.array([np.array(x) for x in prof_list],dtype=float)
    return(prof_list)

def prediction(window_list,info_list_fun):  
    res=[]
    for ar in info_list_fun:
        tot=0
        for li in range(len(ar[1])):
            for el in range(len(ar[1][li])):
                tot=tot+(ar[1][li][el]*window_list[li][el])
        res.append([ar[0],tot])
    #print(res)
    maxi=res[0][1]
    if res[0][1]==0.0 and res[1][1]==0.0 and res[2][1]==0.0:
        return("C")
    else:
        for el in range(len(res)):
            if res[el][1]>=maxi:
                maxi=res[el][1]
                maxi_ss=res[el][0]
        return(maxi_ss)

prediction_df=pd.DataFrame(columns=["Predicted Secondary Structure"])
for prof_name in os.listdir(prof_path):
    #print("\n", prof_name)
    prof_name=prof_name.split(".")
    prof_name=".".join(prof_name[:-1])
    profile=open(prof_path+prof_name+".pssm")
    prof_list,long=reading_profile(profile) #function recall
    prof_ar=add_window_spaces(prof_list,window) #function recall
    predicted=""
    for el in range(long):
        window_list=prof_ar[el:el+window]
        predict_ss=prediction(window_list,info_list_fun[:-1]) #function recall
        predicted=predicted+predict_ss
    #print(prof_name)
    #print(predicted)
    df_row=pd.Series({"Predicted Secondary Structure":predicted},name=prof_name)
    prediction_df=prediction_df.append(df_row)
prediction_df

,Predicted Secondary Structure
4XO2_A,HHHHHHHHHHHHHHHHHHHHHHHHHHHCCCCEEEEHHHHHHHHHHH...
4XP8_A,ECHHHHHCCEEEHCHHHHHHHHHHHHCCCEEEEEEEEECCCEEEEE...
4XPU_A,HHHHHHHHHHHHHHHHHHHHCCCCCCCEEEEEEEEEEEECCCHHHH...
4XPW_A,CCEEEECCCCHEEEEEHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH...
4Y7M_C,HHHHHHHHHHHEEEEEEEEEEECCCCEEEEEECCCCHHHHHHHEEE...
...,...
7BQI_A,HHHHHHHHHHHHHHHHHHHHHHHHCCCEEEEECCCHHHHHHHHHHH...
5D1V_A,HHHHHHHHCCCHHHHHHHHHHHHHEHHHHHHEEEEHHHHHHHHHHH...
5K31_A,EHCCCHHHHHHHHHHHHHHHEEECCCCCCCCCHHEHHHHEEECCCC...
5V89_A,HHHHHHHHHHHHHCCCCCEEEHCCHEEEEEHCCCCCHHHHHHHHHH...


## Support Vector Machines

I will use the **scikit-lear** library. https://scikit-learn.org/stable/

#### SVMs with scikit-learn: https://scikit-learn.org/stable/modules/svm.html

Different implementatios for different problems:
- SVC: Support Vector Classification
- NuSVC: A different mathematical formulation of SVC
- SVR: Support vector Regre

We will focus on multi-class SVC

Training data consist of:
- A matrix **X** contining all example vectors (in rows)
- A vector **y** containing target classes (on for each vector in x)
- **The number of rows in X = the number of elements in y**

I have two options:

- Define **X** as a list of floats and **y** as a list of integers
- Define both **X** and **y** as **numpy** arrays (**X** is bi-dimensional whereas **y** is unidimesional.

In both cases I have to use a **dense representation**.
- Also **zero frequences are reported**

#### SVM for SS prediction

Our **input vectors** are **sequence profile windows**

We can represent one profile **window of size w** as a **vector of dimension w$\times$20**

**Each window is a pont in the w$\times$20 dimensional space**

Window vectors belong to one of the three different classes assigned according to the conformation of the central residue: **1(Helix), 2 (Strand) or 3 (Coil)**

**Multi-class SVM** implemented by LIBSVM (and scikit-learn too) using the **one-against-rest** aproach:
- **Three binary SVMs** are actually trained: **Helix vs Other** and **Coil vs Other**
- In the prediction phase, the predicted class is chosen according to the value of the discrimination function for the three SVMs

I will now extract and create the input for the SVC process.

In [11]:
profile_path="JPred_pssm/"
dssp_path="JPred_dssp/"

In [12]:
%%time
window=17
def reading_files(dssp,profile):
    profile_list=[]
    for line in dssp:
        if line.startswith(">"):
            sec_str1=""
            sec_str=dssp.readline().rstrip()
            for el in sec_str: 
                if el=="-":
                    sec_str1=sec_str1+"C"
                else:
                    sec_str1=sec_str1+el 
    for line in profile:
        if "A" not in line:
            line=line.rstrip()
            line=line.split()[1:]
            profile_list.append(line)
    profile_list=[[float(j) for j in i] for i in profile_list] #COOL
    return(sec_str1,profile_list)

def class_assign(sec_str):
    ss_class=[]
    for ss in sec_str:
        if ss=="H":
            ss_class.append(1)
        if ss=="E":
            ss_class.append(2)
        if ss=="C":
            ss_class.append(3)
    return(ss_class)

def add_window_spaces(profile_list,window):
    for i in range(window//2):
        profile_list=[[0.0 for i in range(20)]]+profile_list
        profile_list.append([0.0 for i in range(20)])
    return(profile_list)

y=[]
X=[]
for filename in os.listdir(profile_path):
    filename=filename.split(".")
    filename=".".join(filename[:-1])
    profile=open(profile_path+filename+".pssm")
    dssp=open(dssp_path+filename+".dssp")
    sec_str,profile_list=reading_files(dssp,profile) #function recall
    ss_class=class_assign(sec_str) #function recall
    profile_list=add_window_spaces(profile_list,window) #function recall
    for el in range(len(ss_class)):
        window_list=profile_list[el:el+window]
        window_list=[el for li in window_list for el in li]
        y.append(ss_class[el])
        X.append(window_list)
print(len(y))
print(len(X))

198803
198803
CPU times: user 7.97 s, sys: 639 ms, total: 8.61 s
Wall time: 9.21 s


### Now with the python scritp present in the folder is possible to perform the 5-fold cross validation on the entire JPred4 dataset

## Evaluation procedure

Evaluating binary classifiers

Main binary scoring indexes:

$$ACC=\frac{TP+TN}{TP+TN+FP+FN}$$

$$Sen=\frac{TP}{TP+FN}$$

$$PPV=\frac{TP}{TP+FP}$$

$$MCC=\frac{TP\times TN-FP\times FN}{\sqrt{(TP+FP)\times(TP+FN)\times(TN+FP)\times(TN+FN)}}$$

#### Multi-class problems

**K-calss confusion matrix**

$p_{ij}$=object in class i predicted in class j

$$Q_k=\frac{p_{11}+p_{22}+...+p_{KK}}{N}$$

#### Binary scores computed from the binary matrix e.g. foe helix

$$Sen_h=\frac{c_H}{c_H+c_H}$$

$$PPV_H=\frac{c_H}{c_H+o_H}$$

$$MCC=\frac{c_H\times n_H-o_H\times u_H}{\sqrt{(c_H+o_H)\times(c_H+u_H)\times(n_H+o_H)\times(n_H+u_H)}}$$

- $c_H$ = $p_{HH}$ = helix resdue correctly predicted in class H (**correct positive**)
- $o_H$ = $p_{EH}+p_{CH}$ = non-helix residues predicted as H (**over-predictions**)
- $u_H$ = $p_{HE}+p_{HC}$ = helix residues as non-H (**under-prediction**)
- $n_H$ = $p_{EE}+p_{CE}+p_{EC}+p_{CC}$ = non-helix residues predicted as non.-H (**correct negative**)

Three-class accuracy ($Q_3$) is computed from 3-class matrix

$$Q_3=\frac{p_{HH}+p_{EE}+p_{CC}}{N}$$

This will be the evaluation of the GOR model trained on the entire JPred class and then tested on the Blind_set

In [219]:
blind_dssp="Blindset_DSSP/"

In [230]:
conf_dict={}

def reading_files(obs_file):
    for line in obs_file:
        if line.startswith(">"):
            obs=obs_file.readline().rstrip()
            return(obs)

def conf_mat(pred,obs,conf_dict):
    for ss in range(len(obs)):
        key=conf_dict.keys()
        if obs[ss]+pred[ss] not in key:
            conf_dict[obs[ss]+pred[ss]]=1
        else: 
            conf_dict[obs[ss]+pred[ss]]+=1
    return(conf_dict)

def accuracy(conf_dict):
    tot=0
    for p in conf_dict:
        tot+=conf_dict[p]
    acc=(conf_dict["HH"]+conf_dict["EE"]+conf_dict["CC"])/tot
    return(acc)

def sensitivity(conf_dict):
    sen_H=conf_dict["HH"]/(conf_dict["HH"]+(conf_dict["HE"]+conf_dict["HC"])) #conf_dict["HE"] --> observed as helixes (H) and predicted ass beta-strand or Coil (P["HC"]) 
    sen_E=conf_dict["EE"]/(conf_dict["EE"]+(conf_dict["EH"]+conf_dict["EC"]))
    sen_C=conf_dict["CC"]/(conf_dict["CC"]+(conf_dict["CE"]+conf_dict["CH"]))
    return(sen_H,sen_E,sen_C)
    
def precision(cond_dict):
    ppv_H=conf_dict["HH"]/(conf_dict["HH"]+(conf_dict["EH"]+conf_dict["CH"])) #conf_dict["EH"]+conf_dict["CH"] --> non-helix residues predicted as H
    ppv_E=conf_dict["EE"]/(conf_dict["EE"]+(conf_dict["HE"]+conf_dict["CE"]))     
    ppv_C=conf_dict["CC"]/(conf_dict["CC"]+(conf_dict["EC"]+conf_dict["HC"]))
    return(ppv_H,ppv_E,ppv_C)

def MCC(cond_dict):
    mcc_H=(cond_dict["HH"]*(cond_dict["EE"]+cond_dict["CE"]+cond_dict["EC"]+cond_dict["CC"])-((cond_dict["EH"]+cond_dict["CH"])*(cond_dict["HE"]+cond_dict["HC"])))/math.sqrt((cond_dict["HH"]+cond_dict["EH"]+cond_dict["CH"])*(cond_dict["HH"]+cond_dict["HE"]+cond_dict["HC"])*(cond_dict["EE"]+cond_dict["CE"]+cond_dict["EC"]+cond_dict["CC"]+cond_dict["EH"]+cond_dict["CH"])*(cond_dict["EE"]+cond_dict["CE"]+cond_dict["EC"]+cond_dict["CC"]+cond_dict["HE"]+cond_dict["HC"]))
    mcc_E=(cond_dict["EE"]*(cond_dict["HH"]+cond_dict["CH"]+cond_dict["HC"]+cond_dict["CC"])-((cond_dict["HE"]+cond_dict["CE"])*(cond_dict["EH"]+cond_dict["EC"])))/math.sqrt((cond_dict["EE"]+cond_dict["HE"]+cond_dict["CE"])*(cond_dict["EE"]+cond_dict["EH"]+cond_dict["EC"])*(cond_dict["HH"]+cond_dict["CH"]+cond_dict["HC"]+cond_dict["CC"]+cond_dict["HE"]+cond_dict["CE"])*(cond_dict["HH"]+cond_dict["CH"]+cond_dict["HC"]+cond_dict["CC"]+cond_dict["EH"]+cond_dict["EC"]))
    mcc_C=(cond_dict["CC"]*(cond_dict["HH"]+cond_dict["EH"]+cond_dict["HE"]+cond_dict["EE"])-((cond_dict["HC"]+cond_dict["EC"])*(cond_dict["CH"]+cond_dict["CE"])))/math.sqrt((cond_dict["CC"]+cond_dict["HC"]+cond_dict["EC"])*(cond_dict["CC"]+cond_dict["CH"]+cond_dict["CE"])*(cond_dict["HH"]+cond_dict["EH"]+cond_dict["HE"]+cond_dict["EE"]+cond_dict["HC"]+cond_dict["EC"])*(cond_dict["HH"]+cond_dict["EH"]+cond_dict["HE"]+cond_dict["EE"]+cond_dict["CH"]+cond_dict["CE"]))
    return(mcc_H,mcc_E,mcc_C)

for index in prediction_df.index:
    pred=prediction_df.loc[index,"Predicted Secondary Structure"]
    obs_file=open(blind_dssp+index+".dssp")
    obs=reading_files(obs_file) #function recall
    conf_dict=conf_mat(pred,obs,conf_dict) #function recall
#print(conf_dict)    
acc=accuracy(conf_dict)
sen_H,sen_E,sen_C=sensitivity(conf_dict)
ppv_H,ppv_E,ppv_C=precision(conf_dict)
mcc_H,mcc_E,mcc_C=MCC(conf_dict)
print("Accuracy: ",acc)
print("- Sensitivity","\t", "for H:",sen_H , end="\t")
print("for E:",sen_E, end="\t")
print("for C:",sen_C)
print("- Precision","\t", "for H:", ppv_H , end="\t")
print("for E:",ppv_E, end="\t")
print("for C:",ppv_C)
print("- MCC" ,"\t\t" ,"for H:", mcc_H , end="\t")
print("for E:",mcc_E, end="\t")
print("for C:",mcc_C)



Accuracy:  0.6377463171991582
- Sensitivity 	 for H: 0.7825829383886256	for E: 0.6917435037720033	for C: 0.44542619542619544
- Precision 	 for H: 0.6642196299275945	for E: 0.5228064618308521	for C: 0.7372043010752688
- MCC 		 for H: 0.5057361334430451	for E: 0.4616530439280952	for C: 0.4093086432120267


## 5-fold cross validation

**Gor-Matrix**

In [34]:
def Gor_method1(pssm_path,dssp_path,train_set,pssm_files):
    def extracter(dssp,pssm):
        for line in dssp:
            if line.startswith(">"):
                sec_str1=""
                sec_str=dssp.readline().rstrip()
                for el in sec_str: 
                    if el=="-":
                        sec_str1=sec_str1+"C"
                    else:
                        sec_str1=sec_str1+el 
        for line in pssm:
            if "A" not in line:
                line=line.rstrip()
                line=line.split()[1:]
                pssm_list.append(line)
        return(sec_str1,pssm_list)

    def window_pssm(pssm_list,sec_str,window,el):
            window_array=pssm_list[el-(window//2):el+window//2+1]
            return(window_array,sec_str[el-window//2])

    def add_window_spaces(pssm_list,window):
        for i in range(window//2):
            pssm_list=[[0.0 for i in range(20)]]+pssm_list
            pssm_list.append([0.0 for i in range(20)])
        pssm_array=np.array([np.array(list) for list in pssm_list])
        pssm_list_float=pssm_array.astype(np.float)
        return(pssm_list_float)

    def fill_gor_mat(window_array,el_sec_str,gor_list_mat):
        for list in range(len(gor_list_mat)):
            if gor_list_mat[list][0]==el_sec_str:
                gor_list_mat[list][1]+=window_array
        gor_list_mat[3][1]+=window_array
        return(gor_list_mat)

    def gor_df_fun(tot_gor_mat,secstr,aa,window):
        index=[]
        gor_df1=pd.DataFrame(tot_gor_mat)
        gor_df1.columns=aa
        for el in secstr:
            for i in range(window):
                index.append("R"+str(i-(window//2))+"_"+el)
        gor_df1.index=index
        return(gor_df1)

    def normalization(gor_list_mat):
        summation=[]
        summation=np.sum(gor_list_mat[3][1],axis=1)
        for array in gor_list_mat:
            for sub in range(len(array[1])):
                array[1][sub]=array[1][sub]/summation[sub]
        return(gor_list_mat)


    aa=['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']
    secstr=["H","E","C","tot"]
    window=17
    gor_list_mat=[]
    for el in secstr:
        gor_list_mat.append([el,np.zeros(((window),len(aa)),float)])
    prob_sec_str={}
    
    for filename in train_set:
        #print(pssm_file)
        if filename not in pssm_files:
            continue
        pssm_list=[]
        pssm=open(pssm_path+filename+".pssm")
        dssp=open(dssp_path+filename+".dssp")
        sec_str,pssm_list=extracter(dssp,pssm) # function recall
        pssm_array=add_window_spaces(pssm_list,window) #function recall
        for el in range(window//2,len(sec_str)+window//2):
            window_array,el_sec_str=window_pssm(pssm_array,sec_str,window,el) #function recall
            gor_list_mat=fill_gor_mat(window_array,el_sec_str,gor_list_mat) #function recall
            keys=prob_sec_str.keys()
            if el_sec_str in keys:
                prob_sec_str[el_sec_str]+=1
            else: 
                prob_sec_str[el_sec_str]=1
    gor_list_mat=normalization(gor_list_mat) #function recall
    tot_gor_mat=np.concatenate((gor_list_mat[0][1],gor_list_mat[1][1],gor_list_mat[2][1],gor_list_mat[3][1]),axis=0)
    gor_df=gor_df_fun(tot_gor_mat,secstr,aa,window) #fucntion recall
    #print("Number of the secondary structures: ",prob_sec_str)
    return(gor_df,prob_sec_str,gor_list_mat)

#gor_df,prob_sec_str,gor_list_mat=Gor_method1(pssm_path,dssp_path,train_set)
#print(gor_list_mat)

**Information function**

In [43]:
def info_matrix(prob_sec_str,gor_list_mat):
    tot_sec_str=0
    for el in prob_sec_str:
        tot_sec_str+=prob_sec_str[el]
    for el in prob_sec_str:
        prob_sec_str[el]=prob_sec_str[el]/tot_sec_str
    #print(prob_sec_str)
    
    info_list_fun=gor_list_mat
    
    #print(info_list_fun)
    for mat in info_list_fun:
        if mat[0]=="tot":
            tot_info_fun=mat[1]

    for ar in (info_list_fun[:-1]):
        for vec in range(len(ar[1])):
            for el in range(len(ar[1][vec])):
                P_ss=prob_sec_str[ar[0]]
                P_join=ar[1][vec][el]
                P_res=tot_info_fun[vec][el]
                ar[1][vec][el]=math.log2(P_join/(P_ss*P_res))
    #print(info_list_fun)
    return(info_list_fun)
    
#info_list_fun=info_matrix(prob_sec_str,gor_list_mat)
#print(info_list_fun)

**Prediction**

In [36]:
def prediction1(jpred_pssm_path,test_set,pssm_files,info_list_fun):
    #import math
    def reading_profile(profile):
        prof_list=[]
        long=0
        for line in profile:
            if "A" not in line:
                #prof_line=[]
                line=line.rstrip()
                array=line.split()[1:]
                prof_list.append(array)
                long+=1
        return(prof_list,long)

    def add_window_spaces(prof_list,window):
        for i in range(window//2):
            prof_list=[[0.0 for i in range(20)]]+prof_list
            prof_list.append([0.0 for i in range(20)])
        prof_list=np.array([np.array(x) for x in prof_list],dtype=float)
        return(prof_list)

    def prediction(window_list,info_list_fun):  
        res=[]
        for ar in info_list_fun:
            tot=0
            for li in range(len(ar[1])):
                for el in range(len(ar[1][li])):
                    tot=tot+(ar[1][li][el]*window_list[li][el])
            res.append([ar[0],tot])
        #print(res)
        maxi=res[0][1]
        if res[0][1]==0.0 and res[1][1]==0.0 and res[2][1]==0.0:
            return("C")
        else:
            for el in range(len(res)):
                if res[el][1]>=maxi:
                    maxi=res[el][1]
                    maxi_ss=res[el][0]
            return(maxi_ss)

    #print(pssm_files)
    prediction_df=pd.DataFrame(columns=["Predicted Secondary Structure"])
    window=17
    for filename in test_set:
        #print(filename)
        if filename not in pssm_files:
            continue
        profile=open(jpred_pssm_path+filename+".pssm")
        prof_list,long=reading_profile(profile) #function recall
        prof_ar=add_window_spaces(prof_list,window) #function recall
        predicted=""
        for el in range(long):
            window_list=prof_ar[el:el+window]
            predict_ss=prediction(window_list,info_list_fun[:-1]) #function recall
            predicted=predicted+predict_ss
        #print(prof_name)
        #print(predicted)
        df_row=pd.Series({"Predicted Secondary Structure":predicted},name=filename)
        prediction_df=prediction_df.append(df_row)
    #print(prediction_df)
    return(prediction_df)

#prediction_df=prediction1(jpred_pssm_path,test_set,pssm_files,info_list_fun)
#prediction_df

**Evaluation**

In [37]:
def evaluation1(prediction_df,jpred_dssp_path):
    
    def reading_files(obs_file):
        for line in obs_file:
            if line.startswith(">"):
                obs=obs_file.readline().rstrip()
                return(obs)

    def conf_mat(pred,obs,conf_dict):
        for ss in range(len(obs)):
            key=conf_dict.keys()
            if obs[ss]+pred[ss] not in key:
                conf_dict[obs[ss]+pred[ss]]=1
            else: 
                conf_dict[obs[ss]+pred[ss]]+=1
        return(conf_dict)

    def accuracy(conf_dict):
        tot=0
        for p in conf_dict:
            tot+=conf_dict[p]
        #print(tot)
        acc=(conf_dict["HH"]+conf_dict["EE"]+conf_dict["CC"])/tot
        return(acc)

    def sensitivity(conf_dict):
        sen_H=conf_dict["HH"]/(conf_dict["HH"]+(conf_dict["HE"]+conf_dict["HC"])) #conf_dict["HE"] --> observed as helixes (H) and predicted ass beta-strand or Coil (P["HC"]) 
        sen_E=conf_dict["EE"]/(conf_dict["EE"]+(conf_dict["EH"]+conf_dict["EC"]))
        sen_C=conf_dict["CC"]/(conf_dict["CC"]+(conf_dict["CE"]+conf_dict["CH"]))
        return(sen_H,sen_E,sen_C)

    def precision(cond_dict):
        ppv_H=conf_dict["HH"]/(conf_dict["HH"]+(conf_dict["EH"]+conf_dict["CH"])) #conf_dict["EH"]+conf_dict["CH"] --> non-helix residues predicted as H
        ppv_E=conf_dict["EE"]/(conf_dict["EE"]+(conf_dict["HE"]+conf_dict["CE"]))     
        ppv_C=conf_dict["CC"]/(conf_dict["CC"]+(conf_dict["EC"]+conf_dict["HC"]))
        return(ppv_H,ppv_E,ppv_C)

    def MCC(cond_dict):
        mcc_H=(cond_dict["HH"]*(cond_dict["EE"]+cond_dict["CE"]+cond_dict["EC"]+cond_dict["CC"])-((cond_dict["EH"]+cond_dict["CH"])*(cond_dict["HE"]+cond_dict["HC"])))/math.sqrt((cond_dict["HH"]+cond_dict["EH"]+cond_dict["CH"])*(cond_dict["HH"]+cond_dict["HE"]+cond_dict["HC"])*(cond_dict["EE"]+cond_dict["CE"]+cond_dict["EC"]+cond_dict["CC"]+cond_dict["EH"]+cond_dict["CH"])*(cond_dict["EE"]+cond_dict["CE"]+cond_dict["EC"]+cond_dict["CC"]+cond_dict["HE"]+cond_dict["HC"]))
        mcc_E=(cond_dict["EE"]*(cond_dict["HH"]+cond_dict["CH"]+cond_dict["HC"]+cond_dict["CC"])-((cond_dict["HE"]+cond_dict["CE"])*(cond_dict["EH"]+cond_dict["EC"])))/math.sqrt((cond_dict["EE"]+cond_dict["HE"]+cond_dict["CE"])*(cond_dict["EE"]+cond_dict["EH"]+cond_dict["EC"])*(cond_dict["HH"]+cond_dict["CH"]+cond_dict["HC"]+cond_dict["CC"]+cond_dict["HE"]+cond_dict["CE"])*(cond_dict["HH"]+cond_dict["CH"]+cond_dict["HC"]+cond_dict["CC"]+cond_dict["EH"]+cond_dict["EC"]))
        mcc_C=(cond_dict["CC"]*(cond_dict["HH"]+cond_dict["EH"]+cond_dict["HE"]+cond_dict["EE"])-((cond_dict["HC"]+cond_dict["EC"])*(cond_dict["CH"]+cond_dict["CE"])))/math.sqrt((cond_dict["CC"]+cond_dict["HC"]+cond_dict["EC"])*(cond_dict["CC"]+cond_dict["CH"]+cond_dict["CE"])*(cond_dict["HH"]+cond_dict["EH"]+cond_dict["HE"]+cond_dict["EE"]+cond_dict["HC"]+cond_dict["EC"])*(cond_dict["HH"]+cond_dict["EH"]+cond_dict["HE"]+cond_dict["EE"]+cond_dict["CH"]+cond_dict["CE"]))
        return(mcc_H,mcc_E,mcc_C)

    conf_dict={}
    for index in prediction_df.index:
        pred=prediction_df.loc[index,"Predicted Secondary Structure"]
        obs_file=open(jpred_dssp_path+index+".dssp")
        obs=reading_files(obs_file) #function recall
        conf_dict=conf_mat(pred,obs,conf_dict) #function recall
    #print(conf_dict)  
    for key in conf_dict.copy():
        if "-" in key:
            new_key=key.replace("-","C")
            conf_dict[new_key]=conf_dict.pop(key)
    #print(conf_dict)
    acc=accuracy(conf_dict)
    sen_H,sen_E,sen_C=sensitivity(conf_dict)
    ppv_H,ppv_E,ppv_C=precision(conf_dict)
    mcc_H,mcc_E,mcc_C=MCC(conf_dict)
    print("Accuracy: ",acc)
    print("- Sensitivity","\t", "for H:",sen_H , end="\t")
    print("for E:",sen_E, end="\t")
    print("for C:",sen_C)
    print("- Precision","\t", "for H:", ppv_H , end="\t")
    print("for E:",ppv_E, end="\t")
    print("for C:",ppv_C)
    print("- MCC" ,"\t\t" ,"for H:", mcc_H , end="\t")
    print("for E:",mcc_E, end="\t")
    print("for C:",mcc_C)
    return(sen_H,sen_E,sen_C,ppv_H,ppv_E,ppv_C,mcc_H,mcc_E,mcc_C,acc)
    
#evaluation1(prediction_df,jpred_dssp_path)

Now That I have defined all these 4 function I can calculate each of the evaluation parameters for all the 5 cross validation (set0, set1, set2, set3, set4)

In [44]:
#import statistics
jpred_pssm_path="JPred_pssm/"
cross_val_path="CV/"
jpred_dssp_path="JPred_dssp/"

sen_tot_H,sen_tot_E,sen_tot_C=[],[],[]
ppv_tot_H,ppv_tot_E,ppv_tot_C=[],[],[]
mcc_tot_H,mcc_tot_E,mcc_tot_C=[],[],[]
accuracy_tot=[]

pssm_files=[]
for pssm_file in os.listdir(jpred_pssm_path):
    pssm_files.append(pssm_file[:-5])
        
cv_set=[]
for file_name in os.listdir(cross_val_path):
    cv_set.append(file_name)
for test in cv_set:
    test_set=[]
    test_ids=open(cross_val_path+test)
    for line in test_ids:
        line=line.rstrip()
        test_set.append(line)
    train_set=[]
    for train in cv_set:
        if train!=test:
            train_ids=open(cross_val_path+train)
            for line in train_ids:
                line=line.rstrip()
                train_set.append(line) 
    print("tested on: ",test)
    gor_df,prob_sec_str,gor_list_mat=Gor_method1(jpred_pssm_path,jpred_dssp_path,train_set,pssm_files)
    info_list_fun=info_matrix(prob_sec_str,gor_list_mat)
    prediction_df=prediction1(jpred_pssm_path,test_set,pssm_files,info_list_fun)
    #print(len(prediction_df))
    #evaluation1(prediction_df,jpred_dssp_path)
    sen_H,sen_E,sen_C,ppv_H,ppv_E,ppv_C,mcc_H,mcc_E,mcc_C,acc=evaluation1(prediction_df,jpred_dssp_path)
    sen_tot_H.append(sen_H)
    sen_tot_E.append(sen_E)
    sen_tot_C.append(sen_C)
    ppv_tot_H.append(ppv_H)
    ppv_tot_E.append(ppv_E)
    ppv_tot_C.append(ppv_C)
    mcc_tot_H.append(mcc_H)
    mcc_tot_E.append(mcc_E)
    mcc_tot_C.append(mcc_C)
    accuracy_tot.append(acc)
print()
print("Total mean and standard deviation for all the three secondary structures:")
print()
print("Accuracy:",statistics.mean(accuracy_tot),"±",statistics.stdev(accuracy_tot))
print()
print("Sensitivity:\t for H",statistics.mean(sen_tot_H),"±",statistics.stdev(sen_tot_H)/math.sqrt(5),"\t for E",statistics.mean(sen_tot_E),"±",statistics.stdev(sen_tot_E)/math.sqrt(5),"\t for C",statistics.mean(sen_tot_C),"±",statistics.stdev(sen_tot_C)/math.sqrt(5))
print("Precision:\t for H",statistics.mean(ppv_tot_H),"±",statistics.stdev(ppv_tot_H)/math.sqrt(5),"\t for E",statistics.mean(ppv_tot_E),"±",statistics.stdev(ppv_tot_E)/math.sqrt(5),"\t for C",statistics.mean(ppv_tot_C),"±",statistics.stdev(ppv_tot_C)/math.sqrt(5))
print("MCC:\t \t for H",statistics.mean(mcc_tot_H),"±",statistics.stdev(mcc_tot_H)/math.sqrt(5),"\t for E",statistics.mean(mcc_tot_E),"±",statistics.stdev(mcc_tot_E)/math.sqrt(5),"\t for C",statistics.mean(mcc_tot_C),"±",statistics.stdev(mcc_tot_C)/math.sqrt(5))    

tested on:  set0
Accuracy:  0.6251482506424194
- Sensitivity 	 for H: 0.7847054770926628	for E: 0.7161552346570397	for C: 0.4424618264786755
- Precision 	 for H: 0.6424864620938628	for E: 0.4784083201446982	for C: 0.7982058047493403
- MCC 		 for H: 0.5225095376245243	for E: 0.43803593271084296	for C: 0.4207203932944134
tested on:  set1
Accuracy:  0.6272471091244777
- Sensitivity 	 for H: 0.8077864133563616	for E: 0.6978571428571428	for C: 0.44401190748797803
- Precision 	 for H: 0.6154731878495449	for E: 0.514442605686776	for C: 0.8052325581395349
- MCC 		 for H: 0.5241478638564095	for E: 0.44816697667458694	for C: 0.425922372432941
tested on:  set2
Accuracy:  0.626651747655584
- Sensitivity 	 for H: 0.809719370294319	for E: 0.7308157831033648	for C: 0.424
- Precision 	 for H: 0.6367061356297093	for E: 0.48673921805723497	for C: 0.8130574384587942
- MCC 		 for H: 0.5381030668227689	for E: 0.4521345798005523	for C: 0.4162317131669378
tested on:  set3
Accuracy:  0.6254109850898433
- Sens

# Other used scripts

In [204]:
profile_path="JPred_pssm/"
dssp_path="JPred_dssp/"

In [21]:
jpred_pssm_path="JPred_pssm/"
cross_val_path="CV/"
jpred_dssp_path="JPred_dssp/"

def input_SVM(test_train_set,pssm_files,jpred_pssm_path,jpred_dssp_path):
    window=17
    def reading_files(dssp,profile):
        profile_list=[]
        for line in dssp:
            if line.startswith(">"):
                sec_str1=""
                sec_str=dssp.readline().rstrip()
                for el in sec_str: 
                    if el=="-":
                        sec_str1=sec_str1+"C"
                    else:
                        sec_str1=sec_str1+el 
                for line in profile:
                    if "A" not in line:
                        line=line.rstrip()
                        line=line.split()[1:]
                        profile_list.append(line)
        profile_list=[[float(j) for j in i] for i in profile_list] #COOL
        return(sec_str1,profile_list)

    def class_assign(sec_str):
        ss_class=[]
        for ss in sec_str:
            if ss=="H":
                ss_class.append(1)
            if ss=="E":
                ss_class.append(2)
            if ss=="C":
                ss_class.append(3)
        return(ss_class)

    def add_window_spaces(profile_list,window):
        for i in range(window//2):
            profile_list=[[0.0 for i in range(20)]]+profile_list
            profile_list.append([0.0 for i in range(20)])
        return(profile_list)

    y=[]
    X=[]
    for filename in test_train_set:
        if filename not in pssm_files:
            continue
        profile=open(jpred_pssm_path+filename+".pssm")
        dssp=open(jpred_dssp_path+filename+".dssp")
        sec_str,profile_list=reading_files(dssp,profile) #function recall
        ss_class=class_assign(sec_str) #function recall
        profile_list=add_window_spaces(profile_list,window) #function recall
        for el in range(len(ss_class)):
            window_list=profile_list[el:el+window]
            window_list=[el for li in window_list for el in li]
            y.append(ss_class[el])
            X.append(window_list)
    #print(len(y))
    #print(len(X))
    return(X,y)

pssm_files=[]
for pssm_file in os.listdir(jpred_pssm_path):
    pssm_files.append(pssm_file[:-5])    
cv_set=[]
for file_name in os.listdir(cross_val_path):
    cv_set.append(file_name)
for test in cv_set:
    test_set=[]
    test_ids=open(cross_val_path+test)
    for line in test_ids:
        line=line.rstrip()
        test_set.append(line)
    train_set=[]
    for train in cv_set:
        if train!=test:
            train_ids=open(cross_val_path+train)
            for line in train_ids:
                line=line.rstrip()
                train_set.append(line) 
#print(test_set)
#print(train_set)
X_test,y_test=input_SVM(test_set,pssm_files,jpred_pssm_path,jpred_dssp_path)
X_train,y_train=input_SVM(train_set,pssm_files,jpred_pssm_path,jpred_dssp_path)
print("test", len(X_test), len(y_test))
print("train", len(X_train), len(y_train))

test 40396 40396
train 158407 158407


In [45]:
#Put here the three binary confusion matrix
dict_mat={
"m_0":[[ 48619,  22368],
 [  9555, 118749]],
"m_1":[[ 17706,  26172],
 [  4786, 150627]],
"m_2":[[73504, 10922],
 [45121, 69744]]
}
li=["0","1","2"]
mcc_tot=[]
sen_tot=[]
ppv_tot=[]
for el in li:
    #mat="m"+"_"+el
    m=dict_mat["m"+"_"+el]    
    mcc=((m[0][0]*m[1][1])-(m[1][0]*m[0][1]))/math.sqrt((m[0][0]+m[1][0])*(m[0][0]+m[0][1])*(m[1][1]+m[1][0])*(m[1][1]+m[0][1]))
    ppv=(m[0][0])/(m[0][0]+m[1][0])
    sen=(m[0][0])/sum(m[0])
    mcc_tot.append(mcc)
    sen_tot.append(sen)
    ppv_tot.append(ppv)
print("MCC:\t",statistics.mean(mcc_tot),"±",statistics.stdev(mcc_tot)/math.sqrt(3))
print("Sen:\t",statistics.mean(sen_tot),"±",statistics.stdev(sen_tot)/math.sqrt(3))
print("PVV:\t",statistics.mean(ppv_tot),"±",statistics.stdev(ppv_tot)/math.sqrt(3))     

MCC:	 0.5373618210105681 ± 0.05284331356513004
Sen:	 0.6530200952340268 ± 0.13578028435424747
PVV:	 0.7475326320565601 ± 0.06546670232946447


In [24]:
print(statistics.mean([0.44590943652629705,0.8624445340863252,0.7398764584763212]),"±",statistics.stdev([0.44590943652629705,0.8624445340863252,0.7398764584763212])/math.sqrt(3))

0.6827434763629812 ± 0.12359005503236356


In [49]:
y_pred=joblib.load("Pred_Blind.joblib")

In [51]:
print(len(y_pred))

20908


In [52]:
profile_path="Blindset_pssm/"
dssp_path="Blindset_DSSP/"

In [54]:
%%time
window=17
def reading_files(dssp,profile):
    profile_list=[]
    for line in dssp:
        if line.startswith(">"):
            sec_str1=""
            sec_str=dssp.readline().rstrip()
            for el in sec_str: 
                if el=="-":
                    sec_str1=sec_str1+"C"
                else:
                    sec_str1=sec_str1+el 
    for line in profile:
        if "A" not in line:
            line=line.rstrip()
            line=line.split()[1:]
            profile_list.append(line)
    profile_list=[[float(j) for j in i] for i in profile_list] #COOL
    return(sec_str1,profile_list)

def class_assign(sec_str):
    ss_class=[]
    for ss in sec_str:
        if ss=="H":
            ss_class.append(1)
        if ss=="E":
            ss_class.append(2)
        if ss=="C":
            ss_class.append(3)
    return(ss_class)

def add_window_spaces(profile_list,window):
    for i in range(window//2):
        profile_list=[[0.0 for i in range(20)]]+profile_list
        profile_list.append([0.0 for i in range(20)])
    return(profile_list)

#i=0
y_true=[]
X_pred=[]
for filename in os.listdir(profile_path):
    #print(filename)
    #i+=1
    filename=filename.split(".")
    filename=".".join(filename[:-1])
    profile=open(profile_path+filename+".pssm")
    dssp=open(dssp_path+filename+".dssp")
    sec_str,profile_list=reading_files(dssp,profile) #function recall
    ss_class=class_assign(sec_str) #function recall
    #print(len(ss_class))
    #print(len(profile_list))
    profile_list=add_window_spaces(profile_list,window) #function recall
    for el in range(len(ss_class)):
        window_list=profile_list[el:el+window]
        window_list=[el for li in window_list for el in li]
        y_true.append(ss_class[el])
        X_pred.append(window_list)
X_pred=np.asarray(X_pred)
y_true=np.asarray(y_true)
#print(i)
print(len(y_true))
print(len(X_pred))
#print(type(y_true))
#print(type(X_pred))

20908
20908
CPU times: user 805 ms, sys: 0 ns, total: 805 ms
Wall time: 831 ms


In [69]:
print(len(y_true))
print(len(y_pred))

20908
20908


In [70]:
conf_mat=confusion_matrix(y_true,y_pred,labels=[1,2,3])

In [71]:
print(conf_mat)

[[5403  120 2917]
 [ 217 2349 2206]
 [ 500  380 6816]]


In [72]:
# conf_mat=[[6093,  151, 2553],
#  [ 284, 2407, 2267],
#  [ 700,  463, 7023]]

def build_2_matrix(m3):
    m2_H=np.array([[0,0],[0,0]])
    m2_E=np.array([[0,0],[0,0]])
    m2_C=np.array([[0,0],[0,0]])

    #2 classes H
    m2_H[0][0]=m3[0][0]
    m2_H[0][1]=m3[0][1]+m3[0][2]
    m2_H[1][0]=m3[1][0]+m3[2][0]
    m2_H[1][1]=m3[1][1]+m3[1][2]+m3[2][1]+m3[2][2]

    #2 classes E
    m2_E[0][0]=m3[1][1]
    m2_E[0][1]=m3[1][0]+m3[1][2]
    m2_E[1][0]=m3[0][1]+m3[2][1]
    m2_E[1][1]=m3[0][0]+m3[0][2]+m3[2][0]+m3[2][2]

    #2 classes C
    m2_C[0][0]=m3[2][2]
    m2_C[0][1]=m3[2][0]+m3[2][1]
    m2_C[1][0]=m3[0][2]+m3[1][2]
    m2_C[1][1]=m3[0][0]+m3[0][1]+m3[1][0]+m3[1][1]

    return m2_H,m2_E,m2_C

def mcc(m):
    d=(m[0][0]+m[1][0])*(m[0][0]+m[0][1])*(m[1][1]+m[1][0])*(m[1][1]+m[0][1])
    return (m[0][0]*m[1][1]-m[0][1]*m[1][0])/math.sqrt(d)

def get_acc(cm):
    #print (sum(cm[0])+sum(cm[1])+sum(cm[2]))
    return float(cm[0][0]+cm[1][1]+cm[2][2])/(sum(cm[0])+sum(cm[1])+sum(cm[2]))

def sen(m):
    return m[0][0]/(sum(m[0]))

def ppv(m):
    return m[0][0]/(m[0][0]+m[1][0])

print("Accuracy:",get_acc(conf_mat))
m2_H,m2_E,m2_C=build_2_matrix(conf_mat)
print()
print("mat 2x2 for H")
print(m2_H)
print("mat 2x2 for E")
print(m2_E)
print("mat 2x2 for C")
print(m2_C)
print()
print("mcc C:",mcc(m2_C),"mcc H:",mcc(m2_H),"mcc E:",mcc(m2_E))
print("sen C:",sen(m2_C),"sen H:",sen(m2_H),"sen E:",sen(m2_E))
print("ppv C:",ppv(m2_C),"ppv H:",ppv(m2_H),"ppv E:",ppv(m2_E))

Accuracy: 0.6967667878324086

mat 2x2 for H
[[ 5403  3037]
 [  717 11751]]
mat 2x2 for E
[[ 2349  2423]
 [  500 15636]]
mat 2x2 for C
[[6816  880]
 [5123 8089]]

mcc C: 0.4851813614200574 mcc H: 0.6282802278114419 mcc E: 0.5642878706423657
sen C: 0.8856548856548857 sen H: 0.6401658767772512 sen E: 0.49224643755238895
ppv C: 0.5709020856018092 ppv H: 0.8828431372549019 ppv E: 0.8244998244998245


In [82]:
mcc_tot=[0.633,0.488,0.801]
print("MCC:\t",statistics.mean(mcc_tot),"±",statistics.stdev(mcc_tot)/math.sqrt(3))

MCC:	 0.6406666666666667 ± 0.09043659534600902
